In [1]:
import json,time
import numpy as np
import pandas as pd
import os, subprocess
import argparse
from collections import OrderedDict
import operator

In [6]:
parser = argparse.ArgumentParser(description='Shape the answer')
parser.add_argument('--nbest_path', type=str,  help='location of nbest_predictions.json')
parser.add_argument('--output_path', type=str,  help='location of nbest_predictions.json')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--nbest_path NBEST_PATH]
                             [--output_path OUTPUT_PATH]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/yanxu/Library/Jupyter/runtime/kernel-dbaa86ed-8efc-4c53-9938-220fbcbe12d8.json


SystemExit: 2

In [7]:
def textrip(text):
    if text=="":
        return text
    if text[-1]==',' or text[-1]=='.' or text[-1]==' ':
        return text[:-1]
    if len(text)>2 and text[0]=='(' and text[-1]==')':
        if text.count('(')==1 and text.count(')')==1:
            return text[1:-1]
    if ('(' in text) and (')' not in text):
        return ""
    if ('(' not in text) and (')' in text):
        return ""
    return text

In [ ]:
#### Checking nbest_BioASQ-test prediction.json
if not os.path.exists(args.nbest_path):
    print("No file exists!\n#### Fatal Error : Abort!")
    raise

#### Reading Pred File
with open(args.nbest_path, "r") as reader:
    test=json.load(reader)

qidDict=dict()
if True:
    for multiQid in test:
        assert len(multiQid)==(24+4) 
        if not multiQid[:-4] in qidDict:
            qidDict[multiQid[:-4]]=[test[multiQid]]
        else :
            qidDict[multiQid[:-4]].append(test[multiQid])
else: 
    qidDict={qid:[test[qid]] for qid in test}    


entryList=[]
entryListWithProb=[]

for qid in qidDict:

    jsonList=[]
    for jsonele in qidDict[qid]: 
        jsonList+=jsonele

    #if not args.multi_output:
    qidDf=pd.DataFrame().from_dict(jsonList)
    
    
    sortedDf=qidDf.sort_values(by='probability', axis=0, ascending=False)

    sortedSumDict=OrderedDict()
    sortedSumDictKeyDict=dict() 

  
    for index in sortedDf.index:
    text = sortedDf.ix[index]["text"]
    text=textrip(text)
    if text=="":
        pass
    elif len(text)>100:
            pass
    elif text.lower() in sortedSumDictKeyDict:
        sortedSumDict[sortedSumDictKeyDict[text.lower()]] += sortedDf.ix[index]["probability"]
    else:
        sortedSumDictKeyDict[text.lower()]=text
        sortedSumDict[sortedSumDictKeyDict[text.lower()]] = sortedDf.ix[index]["probability"]        
    finalSorted=sorted(sortedSumDict.items(), key=operator.itemgetter(1), reverse=True) 
    
    entry={u"type":"factoid", 

    u"id":qid,
    u"ideal_answer":["Dummy"],
    u"exact_answer":[[ans[0]] for ans in finalSorted[:5]],
    
    }
    entryList.append(entry)
    
    entryWithProb={u"type":"factoid", 
    u"id":qid, # must be 24 char
    u"ideal_answer":["Dummy"],
    u"exact_answer":[ans for ans in finalSorted[:20]],
    }
    entryListWithProb.append(entryWithProb)
finalformat={u'questions':entryList}
finalformatWithProb={u'questions':entryListWithProb}

if os.path.isdir(args.output_path):
    outfilepath=os.path.join(args.output_path, "BioASQform_BioASQ-answer.json")
    outWithProbfilepath=os.path.join(args.output_path, "WithProb_BioASQform_BioASQ-answer.json")
else:
    outfilepath=args.output_path
    outWithProbfilepath=args.output_path+"_WithProb"

with open(outfilepath, "w") as outfile:
    json.dump(finalformat, outfile, indent=2)
with open(outWithProbfilepath, "w") as outfile_prob:
    json.dump(finalformatWithProb, outfile_prob, indent=2)
